In [1]:
from floweaver import *
import ipywidgets as widgets
from IPython.display import display, clear_output

size = dict(width=570, height=600)

def load_data(file_name):
    dataset = Dataset.from_csv(file_name, dim_process_filename='new-energy-values-processes.csv')
    partition = dataset.partition('type')
    return dataset, partition

# Hydrogen Taskforce Cases
dataset_neutral, partition_neutral = load_data('neutral-energy-values.csv')
dataset_conservative, partition_conservative = load_data('conservative-energy-values.csv')
dataset_optimistic, partition_optimistic = load_data('optimistic-energy-values.csv')

# FES 2022 cases
dataset_fes22ct, partition_fes22ct = load_data('FES22CT.csv')
dataset_fes22fs, partition_fes22fs = load_data('FES22FS.csv')
dataset_fes22ltw, partition_fes22ltw = load_data('FES22LTW.csv')
dataset_fes22st, partition_fes22st = load_data('FES22ST.csv')

# FES 2021 cases
dataset_fes21ct, partition_fes21ct = load_data('FES21CT.csv')
dataset_fes21fs, partition_fes21fs = load_data('FES21FS.csv')
dataset_fes21ltw, partition_fes21ltw = load_data('FES21LTW.csv')
dataset_fes21st, partition_fes21st = load_data('FES21ST.csv')


sources = ['Green Hydrogen', 'Blue Hydrogen', 'Pink Hydrogen', 'Hydrogen']
uses = ['Hydrogen', 'Road Transport', 'Residential', 'Commercial', 'Industrial', 'Power Generation', 'Rail', 'Shipping', 'Blending',
        'Direct Air Carbon Capture and Storage', 'Transmission Losses', 'Natural Gas', 'Bioresources', 'Nuclear', 'Networked Electricity',
        'Non-Networked Electricity', 'Industry', 'Transportation', 'Power', 'Heating']
excess = ['Excess']
shortfall = ['Shortfall']

nodes = {
    'production_sources': ProcessGroup('type == "source"', Partition.Simple('process', sources), title='Production Sources'),
    'Hydrogen Generation': ProcessGroup(['Hydrogen_Generation'], title='Net Hydrogen\n Generation\n\n\n\n\n'),
    'consumption': ProcessGroup('type == "use"', partition=Partition.Simple('process', uses), title='Consumption'),
    'excess': ProcessGroup('type == "excess"', partition=Partition.Simple('process', excess), title='Excess'),
}

ordering = [
    [['production_sources']],
    [['Hydrogen Generation']],
    [['consumption'], ['excess']],
]

bundles = [
    Bundle('production_sources', 'Hydrogen Generation'),
    Bundle('Hydrogen Generation', 'consumption'),
    Bundle('Hydrogen Generation', 'excess'),
]

palette = {
    'Green Hydrogen': 'green',
    'Blue Hydrogen': 'blue',
}

# Neutral case Sankey
sdd_neutral = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_neutral)
weave(sdd_neutral, dataset_neutral, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

# Conservative case Sankey
sdd_conservative = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_conservative)
weave(sdd_conservative, dataset_conservative, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

# Optimistic Case Sankey
sdd_optimistic = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_optimistic)
weave(sdd_optimistic, dataset_optimistic, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)
output = widgets.Output()

#FES 2022 Consumer Transformation Sankey
sdd_FES22CT = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_neutral)
weave(sdd_FES22CT, dataset_fes22ct, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

#FES 2022 Falling  Short Sankey
sdd_FES22FS = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_neutral)
weave(sdd_FES22FS, dataset_fes22fs, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

#FES 2022 Leading The Way Sankey
sdd_FES22LTW = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_neutral)
weave(sdd_FES22LTW, dataset_fes22ltw, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

#FES 2022 System Transformation Sankey
sdd_FES22ST = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_neutral)
weave(sdd_FES22ST, dataset_fes22st, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

#FES 2021 Consumer Transformation Sankey
sdd_FES21CT = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_neutral)
weave(sdd_FES21CT, dataset_fes21ct, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

#FES 2021 Falling  Short Sankey
sdd_FES21FS = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_neutral)
weave(sdd_FES21FS, dataset_fes21fs, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

#FES 2021 Leading The Way Sankey
sdd_FES21LTW = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_neutral)
weave(sdd_FES21LTW, dataset_fes21ltw, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

#FES 2021 System Transformation Sankey
sdd_FES21ST = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_neutral)
weave(sdd_FES21ST, dataset_fes21st, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)


def on_button_click(button):
    with output:
        clear_output(wait=True)
        if button.description == 'Neutral Case':
            display(weave(sdd_neutral, dataset_neutral, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True, link_label_format=".2s"))
        elif button.description == 'Conservative Case':
            display(weave(sdd_conservative, dataset_conservative, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True, link_label_format=".2s"))
        elif button.description == 'Optimistic Case':
            display(weave(sdd_optimistic, dataset_optimistic, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True, link_label_format=".2s"))
        elif button.description == 'FES 2022 Consumer Transformation Case':
            display(weave(sdd_FES22CT, dataset_fes22ct, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True, link_label_format=".2s"))
        elif button.description == 'FES 2022 Falling Short Case':
            display(weave(sdd_FES22FS, dataset_fes22fs, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True, link_label_format=".2s"))
        elif button.description == 'FES 2022 Leading The Way Case':
            display(weave(sdd_FES22LTW, dataset_fes22ltw, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True, link_label_format=".2s"))
        elif button.description == 'FES 2022 System Transformation Case':
            display(weave(sdd_FES22ST, dataset_fes22st, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True, link_label_format=".2s"))
        elif button.description == 'FES 2021 Consumer Transformation Case':
            display(weave(sdd_FES21CT, dataset_fes21ct, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True, link_label_format=".2s"))
        elif button.description == 'FES 2021 Falling Short Case':
            display(weave(sdd_FES21FS, dataset_fes21fs, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True, link_label_format=".2s"))
        elif button.description == 'FES 2021 Leading The Way Case':
            display(weave(sdd_FES21LTW, dataset_fes21ltw, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True, link_label_format=".2s"))
        elif button.description == 'FES 2021 System Transformation Case':
            display(weave(sdd_FES21ST, dataset_fes21st, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True, link_label_format=".2s"))

          

#Button Definitions
button_neutral = widgets.Button(description="Neutral Case")
button_conservative = widgets.Button(description="Conservative Case")
button_optimistic = widgets.Button(description="Optimistic Case")
button_FES22CT = widgets.Button(description="FES 2022 Consumer Transformation Case")
button_FES22FS = widgets.Button(description="FES 2022 Falling Short Case")
button_FES22LTW = widgets.Button(description="FES 2022 Leading The Way Case")
button_FES22ST = widgets.Button(description="FES 2022 System Transformation Case")
button_FES21CT = widgets.Button(description="FES 2021 Consumer Transformation Case")
button_FES21FS = widgets.Button(description="FES 2021 Falling Short Case")
button_FES21LTW = widgets.Button(description="FES 2021 Leading The Way Case")
button_FES21ST = widgets.Button(description="FES 2021 System Transformation Case")

#Button Press
buttons = [button_neutral, button_conservative, button_optimistic, button_FES22CT, button_FES22FS, button_FES22LTW,
           button_FES22ST, button_FES21CT, button_FES21FS, button_FES21LTW, button_FES21ST]

for button in buttons:
    button.on_click(on_button_click)

display(widgets.HBox([button_conservative, button_neutral,  button_optimistic, button_FES22CT, button_FES22FS, button_FES22LTW, button_FES22ST, button_FES21CT, button_FES21FS, button_FES21LTW, button_FES21ST]))
display(output)

FileNotFoundError: [Errno 2] No such file or directory: 'FES22CT.csv'